In [1]:
import cv2 as cv
import os
captura = cv.VideoCapture('./Data/Original/Video/HDL Viaje al futuro del skate con la primera tabla transparente y luminosa del mundo-KCqLQG8peYY.mp4')
contador = 0
dir_hr = './Data/Original/HR/'
name_dir = 'Patin_360'

while (captura.isOpened()):
    ret, imagen = captura.read()
    if ret == True:
        # print(contador)
        
        if contador >= 400 and contador < 431:
            cv.imwrite(f'{dir_hr}/{name_dir}/img_{contador-400}.png', imagen)
        contador += 1
        if contador > 431:
             break

    
    else: break

In [2]:
import cv2 as cv
import os
original_path = f'./Data/Original/HR/{name_dir}'
list_element = list(os.scandir(original_path))

for image in list_element:
    im = cv.imread(image.path)
    down_points = [im.shape[1]//4, im.shape[0]//4]
    im = cv.resize(im, down_points, interpolation= cv.INTER_LINEAR)
    im = cv.GaussianBlur(im,(0,0), 0.5)
    cv.imwrite(image.path.replace('HR','LR'), im)
    print(image.path.replace('HR','LR'))


In [ ]:
import cv2 as cv
import os
original_path = f'./Data/Original/HR/{name_dir}'
list_element = list(os.scandir(original_path))

for image in list_element:
    im = cv.imread(image.path)
    down_points = [im.shape[1]//4, im.shape[0]//4]
    im = cv.resize(im, down_points, interpolation= cv.INTER_LINEAR)
    im = cv.GaussianBlur(im,(0,0), 0.5)
    cv.imwrite(image.path.replace('HR','LR'), im)
    print(image.path.replace('HR','LR'))


In [ ]:
import cv2 as cv
import numpy as np
import os

final_path = 'Data/Original/LR/city_ooscuro'
final_path = final_path+"/{name_image}"


lut = np.arange(256)
a = 50
b = 1
lut = np.clip((lut-a)*b, 0, 256)
# Execute our function on the cropped car plate image
for image in os.scandir('Data/Original/LR/city'):
    path = image.path
    name = image.name
    image = cv.imread(path)
    image = cv.LUT(image, lut).astype('uint8')
    cv.imwrite(final_path.format(name_image=name), image)

In [ ]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
import os


def lut_chart(image, lut):
    """ Applies a LUT to an image and shows the result.
    Args:
    image: Input image to be modified.
    lut: a 256 elements array representing a LUT, where
    indices index input values, and their content the
    output ones.
    """
    
    # Apply LUT
    im_lut = cv2.LUT(image, lut)
    # Show the initial image
    plt.figure(1)
    plt.subplot(2,2,1)
    plt.title('Original image')
    plt.imshow(image)
    # Show the resultant one
    plt.subplot(2,2,2)
    plt.imshow(im_lut)
    plt.title('LUT applied')
    # Plot the used LUT
    plt.subplot(2,2,3)
    plt.title('LUT')
    plt.plot(lut) # Hint: np.arange() can be useful as first argument to this function
    # And finally, the resultant histogram
    plt.subplot(2,2,4)
    plt.hist(im_lut.ravel(),256,[0,256])
    plt.title('Histogram')
    plt.tight_layout()
    plt.savefig('Images_procesada.png', dpi=1000)
    # plt.show()
    
lut = np.arange(256)
a = 50
b = 1
lut = np.clip((lut-a)*b, 0, 256)
image = cv2.imread('Data/Original/HR/city/0001.png')
lut_chart(image, lut)


In [ ]:
# from TecoGAN.runGan import mycall
# cmd1 = ["python3", "TecoGAN/main.py",
#                     "--cudaID", "0",  # set the cudaID here to use only one GPU
#                     "--output_dir", './Data/Final/Control/Net/test_2',  # Set the place to put the results.
#                     "--summary_dir", './basura' + '/',  # Set the place to put the log.
#                     "--mode", "inference",
#                     "--input_dir_LR", './Data/Original/LR/Test_2',  # the LR directory
#                     # "--input_dir_HR", os.path.join("./HR/", testpre[nn]),  # the HR directory
#                     # one of (input_dir_HR,input_dir_LR) should be given
#                     # "--output_pre", sample_dir,  # the subfolder to save current scene, optional
#                     "--num_resblock", "16",  # our model has 16 residual blocks,
#                     # the pre-trained FRVSR and TecoGAN mini have 10 residual blocks
#                     "--checkpoint", 'TecoGAN/model/TecoGAN',  # the path of the trained model,
#                     "--output_ext", "png"  # png is more accurate, jpg is smaller
#                     ]
# mycall(cmd1).communicate()

In [ ]:
import os
from tkinter import N
import pandas as pd
import re

not_inclide_dir = ['Table']

dir_root = 'Data/Samples'
list_dir = list(filter(lambda x: not x.name in not_inclide_dir, list(os.scandir(dir_root))))
df_measures = pd.DataFrame(columns=['Name','PSNR_00', 'SSIM_00', 'LPIPS_00', 'tOF_00', 'tLP100_00'])
df_time_net = pd.DataFrame(columns=['Name'])
df_time_posporces = pd.DataFrame(columns=['Name'])
df_num_images_bloc = pd.DataFrame(columns=['Name']) 

for file in list_dir:
    name = file.name
    sample_path = os.path.join(file.path, 'Measures')
    sample = os.scandir(sample_path)

    print(file.name)

    for measure in sample:
        if measure.is_dir():
            index_sample = re.match('.*\(([0-9]+)\)', measure.name).group(1)
            measure_path = os.path.join(measure.path, 'metrics.csv')
            df = pd.read_csv(measure_path).drop(columns='Unnamed: 0')
            df = df.loc[df.shape[0]-5]
            df['Name'] = name + '__' + index_sample
            df_measures = df_measures.append(df)
# ========== Time net =========
    timeNet = os.path.join(sample_path, 'Time_net.csv')
    df = pd.read_csv(timeNet, index_col=False)
    df.columns = ['Name','Time_unapply']
    df['Name'] = df['Name'].apply(lambda x: f'Sample_{x}')
    df.index = df['Name']
    df = df.drop(columns='Name').T
    df['Name'] = name
    df_time_net = df_time_net.append(df,ignore_index=True)
# ======== Time posproeces ==========
    time_un_apply = os.path.join(sample_path, 'Time_un_apply.csv')
    df = pd.read_csv(time_un_apply, index_col=False).drop(columns='Unnamed: 0')
    df['Bolock_id'] = df['Bolock_id'].apply(lambda x: f'Bolock_id_{x}')
    df.index = df['Bolock_id']
    df = df.drop(columns='Bolock_id').T
    df['Name'] = name
    aux_df = df[df.index == 'Time']
    df_time_posporces = df_time_posporces.append(aux_df,ignore_index=True)
# ======== Sample posporces ==========
    aux_df = df[df.index == 'Num_images']
    df_num_images_bloc = df_num_images_bloc.append(aux_df,ignore_index=True)


df_measures.to_csv('Data/Samples/Table/measures.csv', sep=';', decimal=',')
df_time_net.to_csv('Data/Samples/Table/time_net.csv', sep=';', decimal=',')
df_time_posporces.to_csv('Data/Samples/Table/time_posporces.csv', sep=';', decimal=',')
df_num_images_bloc.to_csv('Data/Samples/Table/num_images_bloc.csv', sep=';', decimal=',')

In [ ]:
import pandas as pd

df = pd.read_csv('Data/Samples/Table/measures.csv', sep=';', decimal=',').drop(columns=['Unnamed: 0'])
df.index = df.Name
df = df.drop(columns=['Name'])
df = df.T

df_City = df.filter(regex=('City_Test.*_10'))
df_Patin = df.filter(regex=('Patin_Test.*_10'))
df_city_oscuro = df.filter(regex=('city_oscuro_Test.*_10'))
df_Patin_360 = df.filter(regex=('Patin_360_Test.*_10'))

sample_columns = [f'Sample_{y}__{x}' for y in ['control', 'RSS_2_1_10', 'RSS_RS_2_1'] for x in range(1, 11)]
df_City.columns = sample_columns
df_Patin.columns = sample_columns
df_city_oscuro.columns = sample_columns
df_Patin_360.columns = sample_columns

df_City.index = list(map(lambda x: f'{x[:-3]}_citi', df_City.index.to_list()))
df_Patin.index = list(map(lambda x: f'{x[:-3]}_patin', df_Patin.index.to_list()))
df_city_oscuro.index = list(map(lambda x: f'{x[:-3]}_citiOscuro', df_city_oscuro.index.to_list()))
df_Patin_360.index = list(map(lambda x: f'{x[:-3]}_patin360', df_Patin_360.index.to_list()))

from sklearn.preprocessing import MinMaxScaler
import numpy as np

def Normalices(table):
    df_copi = table.T
    df_copi.index = list(map(lambda x: f'{x}_Norma', df_copi.index.to_list()))
    for column in df_copi.columns.to_list():
        df_aux = pd.to_numeric(df_copi[column])
        new_values=[]
        for x in range(10, 40, 10):
            new_values += (list((MinMaxScaler().fit_transform(df_aux.iloc[x-10:x].values.reshape(-1,1))).flat))
        df_copi[column] = new_values
        df_copi[column]=df_copi[column]
    return df_copi.T



df_City = Normalices(df_City)
df_Patin = Normalices(df_Patin)
df_city_oscuro = Normalices(df_city_oscuro)
df_Patin_360 = Normalices(df_Patin_360)

df_aux = df_City
df_aux = df_aux.append(df_Patin)
df_aux = df_aux.append(df_city_oscuro)
df_aux = df_aux.append(df_Patin_360)

df_aux.sort_index(inplace=True)

df_aux.T.to_csv('Data/Samples/Table/Measures_Norma_10.csv', sep=';', decimal=',')